In [1]:
import sys
sys.path.append('../')

import tquant as tq


import tensorflow as tf
from datastore import *
import matplotlib.pyplot as plt
import QuantLib as ql

def get_ql_curve(curve_tq, evaluation_date):
    evaluation_date_ql = ql.Date.from_date(evaluation_date)
    dates_ql = [evaluation_date_ql + d for d in curve_tq._pillar_days]
    rates_ql = [r.numpy() for r in curve_tq._rates]
    curve_ql = ql.ZeroCurve(dates_ql, rates_ql, ql.ActualActual(ql.ActualActual.ISDA), ql.TARGET())
    curve_ql.enableExtrapolation()
    return curve_ql


In [2]:
# from abc import ABC, abstractmethod
# from tensorflow.python.framework import dtypes

# class StochasticProcess(ABC):
#     def __init__(self):
#         pass

#     @abstractmethod
#     def size(self):
#         pass
    
#     @property
#     def factors(self):
#         return self.size()
    
#     @abstractmethod
#     def initial_values(self):
#         pass

#     @abstractmethod
#     def drift(self, t0, x0, dt):
#         pass

#     @abstractmethod
#     def diffusion(self, t0, x0, dt):
#         pass

#     def expectation(self, t0, x0, dt):
#         """ aggiungere euler discretization"""
#         return x0 + self.drift(t0, x0, dt) 

#     def std_deviation(self, t0, x0, dt):
#         return self.diffusion(t0, x0, dt)
    
#     def evolve(self, t0, x0, dt, dw):
#         return self.expectation(t0=t0, x0=x0, dt=dt) + self.std_deviation(t0=t0, x0=x0, dt=dt)*dw

# class OrnsteinUhlenbeckProcess(StochasticProcess):
#     def __init__(self, mr_speed, volatility, x0=0.0, level=0.0):
#         self._x0 =  tf.Variable(x0, dtype=dtypes.float64)
#         self._mr_speed =  tf.Variable(mr_speed, dtype=dtypes.float64)
#         self._volatility =  tf.Variable(volatility, dtype=dtypes.float64)
#         self._level =  level#tf.Variable(level, dtype=dtypes.float64)

#     def size(self):
#         return 1
    
#     def initial_values(self):
#         return self.x0

#     def drift(self, x):
#         return self._mr_speed * (self._level - x)

#     def diffusion(self):
#         return self._volatility

#     def expectation(self, x0, dt, t0=None):
#         """ overload di euler """
#         return self._level + (x0 - self._level) * tf.exp(-self._mr_speed * dt)

#     def std_deviation(self, dt, t0=None, x0=None):
#         """ parametri none per rispettare interfaccia"""
#         return tf.sqrt(self.variance(dt))

#     def variance(self, dt):
#         return 0.5*self._volatility**2 / self._mr_speed * (1 - tf.exp(-2 * self._mr_speed * dt))
    
#     @property
#     def x0(self):
#         return self._x0.numpy()

#     @property
#     def mr_speed(self):
#         return self._mr_speed.numpy()

#     @property
#     def volatility(self):
#         return self._volatility.numpy()

#     @property
#     def level(self):
#         return self._level

# class HullWhiteProcess(StochasticProcess):
#     def __init__(self,
#                  term_structure: tq.RateCurve,
#                  a: float,
#                  sigma: float):
#         self._process = OrnsteinUhlenbeckProcess(mr_speed=a, volatility=sigma, x0=term_structure.inst_fwd(0))
#         self._a = tf.Variable(a, dtype=dtypes.float64)
#         self._sigma = tf.Variable(sigma, dtype=dtypes.float64)
#         self._term_structure = term_structure

#     def size(self):
#         return self._process.size()
    
#     def initial_values(self):
#         return self._process.x0
    
#     @property
#     def a(self):
#         return self._a.numpy()

#     @property
#     def sigma(self):
#         return self._sigma.numpy()

#     @property
#     def x0(self):
#         return self._process.x0.numpy()
    
#     def drift(self, t, x):
#         alpha_drift = self._sigma**2 / (2 * self._a) * (1 - tf.exp(-2 * self._a * t))
#         shift = 0.0001
#         f = self._term_structure.forward_rate(t, t)
#         fup = self._term_structure.forward_rate(t + shift, t + shift)
#         f_prime = (fup - f) / shift
#         alpha_drift += self._a * f + f_prime
#         return self._process.drift(t, x) + alpha_drift

#     def diffusion(self, t, x):
#         return self._process.diffusion(t, x) # perché diffusion ha (t,x) se sul c++ ritorna sempre volatility_?

#     def expectation(self, t0, x0, dt):
#         return (self._process.expectation(x0=x0, dt=dt) 
#                 + self.alpha(t0 + dt) - self.alpha(t0) * tf.exp(-self._a * dt))

#     def std_deviation(self, dt, t0=None, x0=None):
#         return self._process.std_deviation(dt=dt)

#     def variance(self, dt):
#         return self._process.variance(dt)

#     def alpha(self, t):
#         if self.a > 1e-10:
#             alfa = (self._sigma / self._a) * (1 - tf.exp(-self._a * t))
#         else:
#             alfa = self._sigma * t
#         alfa = 0.5 * alfa**2
#         alfa += self._term_structure.inst_fwd(t)
#         return alfa

# # class for hosting simulation grid (dates, times)
# class Grid:
#     def __init__(self, startDate, endDate, tenor):
#         # create date schedule, ignore conventions and calendars
#         self.schedule = ql.Schedule(startDate, endDate, tenor, ql.NullCalendar(), 
#             ql.Unadjusted, ql.Unadjusted, ql.DateGeneration.Forward, False)
#         self.dayCounter = ql.Actual365Fixed()
#         self.tenor = tenor
#     def GetDates(self):
#         # get list of scheduled dates
#         dates = []
#         [dates.append(self.schedule[i]) for i in range(self.GetSize())]
#         return dates            
#     def GetTimes(self):
#         # get list of scheduled times
#         times = []
#         [times.append(self.dayCounter.yearFraction(self.schedule[0], self.schedule[i])) 
#             for i in range(self.GetSize())]
#         return times
#     def GetMaturity(self):
#         # get maturity in time units
#         return self.dayCounter.yearFraction(self.schedule[0], self.schedule[self.GetSteps()])
#     def GetSteps(self):
#         # get number of steps in schedule
#         return self.GetSize() - 1    
#     def GetSize(self):
#         # get total number of items in schedule
#         return len(self.schedule)    
#     def GetTimeGrid(self):
#         # get QuantLib TimeGrid object, constructed by using list of scheduled times
#         return ql.TimeGrid(self.GetTimes(), self.GetSize())
#     def GetDt(self):
#         # get constant time step
#         return self.GetMaturity() / self.GetSteps()
#     def GetTenor(self):
#         # get grid tenor
#         return self.tenor

# Settings

In [3]:
tq.Settings.evaluation_date = date(2024, 4, 30)

calendar = tq.TARGET()
daycounter = tq.DayCounter(tq.DayCounterConvention.Actual360)
evaluation_date = tq.Settings.evaluation_date

rates = ir_eur_curve_estr['quote'].values
times = ir_eur_curve_estr['daycount'].values/365


rate_curve = tq.RateCurve(evaluation_date, times, rates, 'LINEAR')
eur6m_index = tq.IborIndex(calendar, 6, tq.TimeUnit.Months, tq.Currency.EUR)

# Test QL

In [4]:
a = 0.03
sigma = 0.2
#TQ
hw_tq = tq.HullWhiteProcess(rate_curve, a, sigma)
# QL
rate_curve_ql = get_ql_curve(rate_curve, evaluation_date)
hrate_curve_ql = ql.YieldTermStructureHandle(rate_curve_ql)
hw_ql = ql.HullWhiteProcess(hrate_curve_ql, a, sigma)


In [5]:
t0 = 2
x0 = 0.01
dt = 3.3

print("tq drift: ", hw_tq.expectation(x0=x0, dt=dt, t0=t0).numpy() )
print("ql drift: ", hw_ql.expectation(t0, x0, dt) ) # differenza data dall'inst. forward
print("*"*30)
print("tq diffusion: ", hw_tq.std_deviation(x0=x0, dt=dt, t0=t0).numpy() )
print("ql diffusion: ", hw_ql.stdDeviation(t0, x0, dt) )

tq drift:  0.42119516645478117
ql drift:  0.421211266538131
******************************
tq diffusion:  0.34605408716188574
ql diffusion:  0.34605408716188574


In [6]:
dw = np.random.normal()
print("QL evolve: ", hw_ql.evolve(t0, x0, dt, dw))
print("TQ evolve: ", hw_tq.evolve(t0, x0, dt, dw).numpy())

QL evolve:  0.6535155393425665
TQ evolve:  0.6534994392592166


# Implementazione tq hull white

In [7]:
# class GaussianPathGenerator:
#     def __init__(self,
#                  process: tq.StochasticProcess,
#                  time_grid: list[float]) -> None:
#         self._process = process
#         self._time_grid = time_grid
#         self._generator = None #TODO implementazione random numbers

#     def simulate(self,
#                  n_paths: int) -> tf.Tensor:
#         path = []
#         path.append(tf.fill((n_paths,),
#                                 value= self._process.initial_values()))
#         for i in range(1, len(self._time_grid)):
#             w = tf.random.normal(shape=(n_paths,), mean=0, stddev=1, dtype= tf.float64)
#             s = self._time_grid[i-1]
#             t = self._time_grid[i]
#             dt = t-s
#             x_t = self._process.evolve(s, path[i-1], dt, w)
#             path.append(x_t)
#         path_tensor = tf.stack(path, axis=1)
#         return path_tensor


In [8]:
process = tq.HullWhiteProcess(rate_curve, a, sigma)
time_grid = np.linspace(start=0.0, stop=10, num=60, retstep=False)
kernel = tq.GaussianPathGenerator(hw_tq, time_grid)

In [9]:
kernel.simulate(n_paths=100000)

<tf.Tensor: shape=(100000, 60), dtype=float64, numpy=
array([[ 0.03906133,  0.02749389, -0.00796917, ...,  1.8378611 ,
         1.96137252,  1.88649841],
       [ 0.03906133,  0.03882495, -0.12797137, ...,  0.73683127,
         0.75222466,  0.79651335],
       [ 0.03906133,  0.03115385, -0.03290972, ...,  1.49475579,
         1.5053906 ,  1.67323518],
       ...,
       [ 0.03906133, -0.03665279,  0.05350294, ...,  2.48437322,
         2.53626651,  2.49049699],
       [ 0.03906133, -0.02052662,  0.05416424, ...,  1.80446699,
         1.7559083 ,  1.85681209],
       [ 0.03906133,  0.06652872,  0.12182186, ...,  1.00545399,
         1.05296104,  1.10653549]])>

## QL

In [10]:
evaluation_date_ql = ql.Date.from_date(evaluation_date)
simulation_end = evaluation_date_ql + ql.Period("20Y")
gridStepPeriod = ql.Period(3, ql.Months)
timeGrid = Grid(evaluation_date_ql, simulation_end, gridStepPeriod)
times = np.array(timeGrid.GetTimes())
dates = np.array(timeGrid.GetDates())

process = hw_ql
Npaths = 100

timestep_len= len(times[1:])
dimension = process.factors()

rng = ql.UniformRandomSequenceGenerator(dimension * timestep_len, ql.UniformRandomGenerator())
sequenceGenerator = ql.GaussianRandomSequenceGenerator(rng)
pathGenerator = ql.GaussianMultiPathGenerator(process, list(times), sequenceGenerator, False)

# singolo path QL-like
generator = sequenceGenerator

sequence = generator.nextSequence().value() # per la timegrid data
m = process.size()
n = process.factors()
asset = list(process.initialValues())
time_grid = times
dt = times[1:]-times[:-1]

path = np.zeros((m, len(time_grid)))
for j in range(m):
    path[j,0] = asset[j]

for i in range(1, path.size):
    offset = (i-1)/n
    offset_int = int(offset)
    temp = np.array(sequence[offset_int:offset_int+n])
    pass
    path[j,i] = process.evolve(time_grid[i], asset[0], dt[i-1], temp[0])
    
# multipli path QL-like
gen_paths = np.zeros((dimension, timestep_len+1, Npaths))
for i in range(Npaths):
    samplePath = pathGenerator.next()
    values = samplePath.value() 

    for j in range(dimension):
        gen_paths[j, :, i] = values[0]

NameError: name 'Grid' is not defined

In [ ]:
# implementazione tq


# OLD

In [ ]:
hw = tq.HullWhiteProcess(a, sigma, rate_curve)
hw_kernel = tq.GaussianRateKernel1D(hw)

# simulate short rate and discount curve 
paths = int(1e6) 
r_tensor, zb_tensor, grid = hw_kernel.rate_tensor(paths,
                                                  last_grid_time=1,
                                                  time_steps=2) 
# Reshape the tensor and create a discount curve object
reshaped_tensor = tf.reshape(zb_tensor, shape=(zb_tensor.shape[1], paths, 1))
rate_curve_tensor = tq.RateCurve.from_zcb(evaluation_date, times, reshaped_tensor, "LINEAR")

In [ ]:
rate_curve_tensor.discount(2.0)

<tf.Tensor: shape=(1000000, 1), dtype=float64, numpy=
array([[0.99560683],
       [1.00017447],
       [0.98124546],
       ...,
       [0.99541913],
       [0.99054788],
       [0.99026692]])>